In [ ]:
!git clone https://github.com/LukaMelinc/NMRV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install necessary libraries
!pip install ultralytics -q

In [ ]:
from ultralytics import YOLO
import torch
import torch.optim as optim

print("loading YOLOv11")
# Load the pre-trained YOLOv11 model
model = YOLO('yolov8m.pt')

print("loading YOLOv11 done")

model.train(
        amp=False,
        data="/content/NMRV/Seminar/data_thermal.yaml",           # File za dataset
        imgsz=736,                  # max. velikost ki jo zahtevamo
        epochs=2,                 # Št. epoch
        batch=2,                    # Velikost batcha
        lr0 = 0.001,                # Initial learning rate
        lrf=0.01,                   # Learning Rate Factor
        cos_lr = False,             # linearno ali kosinusno zmanjšanje
        patience=20,                # Early stop trigger
        device="cuda",              # Rabi GPU
        project="yolo_ladica",      # Directory za training loge
        name="OvireNaVodi",         # Sub-folder name
        workers=1,                  # Št. corov za loadat podatke
        show=True,                  # Prikaži rezultate
        save=True,                  # Shrani naučene uteži

        # Augmentacija
        hsv_h=0.2,                  # Hue
        hsv_s=0.2,                  # Saturation
        hsv_v=0.2,                  # Brightnes
        degrees=15,                 # Rotacija
        translate=0.1,              # Translacija
        scale=0.1,                  # Skaliranje
        fliplr=0.5,                 # Flipanje horizontalno
    )

In [ ]:
# Test the model on the test dataset
test_results = model.val(
    data="/content/NMRV/Seminar/data_thermal.yaml",  # Path to the dataset configuration file
    imgsz=640,                  # Image size for evaluation
    device="cuda",              # Use GPU for evaluation
    split='test'                # Specify the test split
)

def calculate_f1_score(precision, recall):
    if precision + recall > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0.0

# Print the test results
print("Test Results:")
print(test_results)

# Extract precision and recall from results_dict
results_dict = test_results.results_dict
precision = results_dict.get('metrics/precision(B)', 0)
recall = results_dict.get('metrics/recall(B)', 0)

# Calculate F1 score
f1_score = calculate_f1_score(precision, recall)

# Print the F1 score
print(f"F1 Score: {f1_score:.4f}")
